# BIG DATA PROJECT

# Design a streaming system using PySpark to perform different data aggregation analyses using SQL queries for  any given data 

## BY Pratishtha Sharma (IDS2022004) and Gomtesh Jain (IDS2022006)


#### Install the needful packages


In [1]:
!pip install pyspark

#### import needful lib

In [1]:
import pandas as pd

## PANDAS

### we are working with pandas bcz to get basic idea of data and checking for any null values

#### Read the CSV file

In [2]:
df_pd= pd.read_csv("data/paysim.csv", header=0)


#### Shape = row ,col

In [3]:
df_pd.shape

(6362620, 11)

In [4]:
df_pd.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [5]:
df_pd.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [6]:
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


### no null values 

# spark -----


#### import needful lib

In [1]:
#%load_ext nb_black
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
spark = SparkSession.builder.getOrCreate()

22/11/17 02:15:36 WARN Utils: Your hostname, Gomteshs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.17.14.180 instead (on interface en7)
22/11/17 02:15:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/17 02:15:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Read the CSV file

In [2]:
df= spark.read.csv("data/paysim.csv", header=True, inferSchema=True)


In [3]:
df.columns

['step',
 'type',
 'amount',
 'nameOrig',
 'oldbalanceOrg',
 'newbalanceOrig',
 'nameDest',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'isFlaggedFraud']

In [4]:
df=df.drop("isFraud" , "isFlaggedFraud")

In [5]:
df.show(2)

+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|   type| amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+
|   1|PAYMENT|9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|
|   1|PAYMENT|1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|
+----+-------+-------+-----------+-------------+--------------+-----------+--------------+--------------+
only showing top 2 rows



#### Step maps a unit of time in the real world. In this case 1 step is 1 hour of time.So we can assume for this example that we have another job that runs every hour and gets all the transactions in that time frame.

In [6]:
df.groupBy("step").count().show(3)

+----+-----+
|step|count|
+----+-----+
|  31|   12|
|  34|30904|
|  28|    4|
+----+-----+
only showing top 3 rows



#### We can therefore save the output of that job by filtering on each step and saving it to a separate file.

In [7]:
#%% time
#steps = df.select("step").distinct().collect()
#for step in steps[:]:
 #   _df = df.where(f"step = {step[0]}")
  #  #by adding coalesce(1) we save the dataframe to one file
   # _df.coalesce(1).write.mode("append").option("header", "true").csv("data/paysim")

In [8]:
!cd data/paysim/ && ls

_SUCCESS
part-00000-011b3b16-8563-4a2b-9db0-9239fea478bf-c000.csv
part-00000-01a20f08-3eee-483b-8e0f-fc94833f5bb8-c000.csv
part-00000-01bc98f4-d413-415f-b5fe-98978d545d20-c000.csv
part-00000-024e0567-499a-413a-a7da-afb9088ef644-c000.csv
part-00000-02518c4b-18e6-48b2-8eae-de333885fad0-c000.csv
part-00000-02cf0495-97ab-46f5-a18e-ee847ea36848-c000.csv
part-00000-03a7a653-c5b2-47bf-8b47-8d24cf4911ea-c000.csv
part-00000-03f66ddc-7f7c-42c6-9bea-74c0e2dd7624-c000.csv
part-00000-041e4e17-f17b-476e-a7e7-5803a5cacd17-c000.csv
part-00000-044f7692-1cb9-4da7-b1d2-c341652ae5a6-c000.csv
part-00000-0463e27b-4923-4e41-b67e-3f7093983a15-c000.csv
part-00000-04b578b9-20bc-4ea4-8dd8-fdca15fefe49-c000.csv
part-00000-0523e458-7165-40ac-9a1b-12b4cdb3a8a4-c000.csv
part-00000-0529095f-45c7-400c-aa43-cd1ca2c8c93f-c000.csv
part-00000-05a507b4-828c-4b68-9ecd-538ef0a404a5-c000.csv
part-00000-05cc0c42-2d66-46b0-a83c-c039479ed3a1-c000.csv
part-00000-05ee10a9-7d3d-4e4c-84ce-c796a12c7bb3-c000.csv
part-00000-0691324d-fe

In [9]:
part = spark.read.csv(
    "data/paysim/part-00000-ff6abc33-7c9d-4735-9bd3-40da70d29797-c000.csv",header=True,inferSchema=True)

In [10]:
part.show()

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+
| 176|TRANSFER|106642.43|C2073914981|    106642.43|           0.0|C1753584812|           0.0|           0.0|
| 176|CASH_OUT|106642.43| C971898032|    106642.43|           0.0|C1932810385|        4622.5|     111264.92|
| 176|TRANSFER|102229.91|C1177644570|    102229.91|           0.0|C1344630204|           0.0|           0.0|
| 176|CASH_OUT|102229.91|C2034388364|    102229.91|           0.0| C949501581|           0.0|     102229.91|
| 176|TRANSFER|351332.16|C1773992583|    351332.16|           0.0|C1305245838|           0.0|           0.0|
| 176|CASH_OUT|351332.16|C1071813500|    351332.16|           0.0|C2129197098|           0.0|     351332.16|
| 176|TRANSFER| 455

In [11]:
part.groupBy( "step").count().show()

+----+-----+
|step|count|
+----+-----+
| 176|   10|
+----+-----+



#### Let's create a streaming version of this input, we'll read each file one by one as if it was a stream.

In [12]:
dataSchema = part.schema

In [13]:
dataSchema

StructType([StructField('step', IntegerType(), True), StructField('type', StringType(), True), StructField('amount', DoubleType(), True), StructField('nameOrig', StringType(), True), StructField('oldbalanceOrg', DoubleType(), True), StructField('newbalanceOrig', DoubleType(), True), StructField('nameDest', StringType(), True), StructField('oldbalanceDest', DoubleType(), True), StructField('newbalanceDest', DoubleType(), True)])

### maxFilesPerTrigger allows you to control how quickly Spark will read all of the files in the folder.In this example we're limiting the flow of the stream to one file per trigger.

In [14]:
streaming =(
  spark.readStream.schema(dataSchema)
  .option("maxFilesPerTrigger", 1)
  .csv("data/paysim/")
)

#### Let's set up a transformation.
#### apply aggrigate function on amount 

In [15]:
#dest_count = streaming.groupBy( "nameDest").count() .orderBy(F.desc("count"))
dest_count = streaming.agg({"amount": "count"})

In [16]:
#help(streaming)

In [17]:
#dest_count

In [18]:
#_df= spark.sql(
#    "SELECT * FROM dest_count "
#    )
#_df.show()

Now that we have our transformation,we need to specify an output sink for the results.For this example,we're going to write to a memory sink which keeps the results in memory. We also need to define how Spark will output that data. In this example,we'll use the complete output mode(rewriting all of the keys along with their counts after every trigger). A In this example we won't include activityQuery.await Termination()because it is required only to prevent the driver process from terminating when the stream is active. So in order to be able to run this locally in a notebook we won't include it.

In [19]:
dest_count = streaming.agg({"amount": "count"})

In [20]:
activityQuery = (
  dest_count.writeStream.queryName( "dest_count")
  .format("memory")
  .outputMode("complete")
  .start()
)

#include this in production
#activityQuery.await Termination()

import time
#"SELECT * FROM dest_counts WHERE nameDest != 'nameDest' AND count >= 2"

for x in range(50):
    _df= spark.sql(
    "Select * From dest_count "
    )
    #if _df.count() > 0:
    _df.show()
    time.sleep(0.5)

22/11/17 02:16:14 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/yb/c1_9y4655151dq56_3k140br0000gn/T/temporary-f140e10c-6b37-43d3-a445-441fcab1e2f6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/17 02:16:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
+-------------+
|count(amount)|
+-------------+
+-------------+

+-------------+
|count(amount)|
+-------------+
+-------------+

+-------------+
|count(amount)|
+-------------+
|        30916|
+-------------+

+-------------+
|count(amount)|
+-------------+
|        30920|
+-------------+

+-------------+
|count(amount)|
+-------------+
|        31401|
+-------------+

+-------------+
|count(amount)|
+-------------

KeyboardInterrupt: 

### Check if stream is active

In [21]:
spark.streams.active[0].isActive

True

In [22]:
activityQuery.status

{'message': 'Getting offsets from FileStreamSource[file:/Users/gomteshjain/Documents/bigdata/data/paysim]',
 'isDataAvailable': True,
 'isTriggerActive': True}

### If we want to turn off the stream we'll run activityQuery.stop()to reset the query for testing purposes.

In [28]:
activityQuery.stop()

22/11/17 02:16:50 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@18542450 is aborting.
22/11/17 02:16:50 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@18542450 aborted.
22/11/17 02:16:50 WARN TaskSetManager: Lost task 0.0 in stage 226.0 (TID 238) (172.17.14.180 executor driver): TaskKilled (Stage cancelled)


## Checking for remaing Aggregate Functions

## AVG

In [24]:
dest_count = streaming.agg({"amount": "avg"})

In [25]:
activityQuery = (
  dest_count.writeStream.queryName( "dest_count")
  .format("memory")
  .outputMode("complete")
  .start()
)
import time
for x in range(50):
    _df= spark.sql(
    "Select * From dest_count "
    )
    _df.show()
    time.sleep(0.5)

22/11/17 02:16:35 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/yb/c1_9y4655151dq56_3k140br0000gn/T/temporary-dc500d8d-af5d-482e-b932-0c369bef7551. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/17 02:16:35 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
+-----------+
|avg(amount)|
+-----------+
+-----------+

+-----------+
|avg(amount)|
+-----------+
| 1077260.67|
+-----------+

+------------------+
|       avg(amount)|
+------------------+
|178903.53863551214|
+------------------+

+------------------+
|       avg(amount)|
+------------------+
|178109.85849681238|
+------------------+

+----------------+
|     avg(amount)|
+----------------+
|182822.024307074|
+--

KeyboardInterrupt: 

## MIN

In [26]:
dest_count = streaming.agg({"amount": "min"})

In [29]:
activityQuery = (
  dest_count.writeStream.queryName( "dest_count")
  .format("memory")
  .outputMode("complete")
  .start()
)
import time
for x in range(50):
    _df= spark.sql(
    "Select * From dest_count "
    )
    _df.show()
    time.sleep(0.5)

22/11/17 02:16:57 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/yb/c1_9y4655151dq56_3k140br0000gn/T/temporary-d3e6d788-cb42-4cf5-b81f-465e7370077a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/17 02:16:57 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
+-----------+
|min(amount)|
+-----------+
+-----------+

+-----------+
|min(amount)|
+-----------+
|   21516.38|
+-----------+

+-----------+
|min(amount)|
+-----------+
|       0.87|
+-----------+

+-----------+
|min(amount)|
+-----------+
|       0.87|
+-----------+

+-----------+
|min(amount)|
+-----------+
|       0.87|
+-----------+

+-----------+
|min(amount)|
+-----------+
|       0.68|
+-----------+



KeyboardInterrupt: 

## MAX

In [48]:
dest_count = streaming.agg({"amount": "max"})

In [49]:
activityQuery = (
  dest_count.writeStream.queryName( "dest_count")
  .format("memory")
  .outputMode("complete")
  .start()
)
import time
for x in range(50):
    _df= spark.sql(
    "Select * From dest_count "
    )
    _df.show()
    time.sleep(0.5)

22/11/02 15:30:49 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/yb/c1_9y4655151dq56_3k140br0000gn/T/temporary-a6d855d0-fc20-41a6-8f45-41d13c0b9f69. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/02 15:30:49 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
+-----------+
|max(amount)|
+-----------+
+-----------+

+-----------+
|max(amount)|
+-----------+
| 5021186.54|
+-----------+

+-----------+
|max(amount)|
+-----------+
| 5021186.54|
+-----------+

+-----------+
|max(amount)|
+-----------+
| 5021186.54|
+-----------+

+-----------+
|max(amount)|
+-----------+
| 5021186.54|
+-----------+

+-----------+
|max(amount)|
+-----------+
| 5021186.54|
+-----------+

+------

KeyboardInterrupt: 

## SUM

In [51]:
dest_count = streaming.agg({"amount": "sum"})

In [52]:
activityQuery = (
  dest_count.writeStream.queryName( "dest_count")
  .format("memory")
  .outputMode("complete")
  .start()
)
import time
for x in range(50):
    _df= spark.sql(
    "Select * From dest_count "
    )
    _df.show()
    time.sleep(0.5)

22/11/02 15:31:32 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/yb/c1_9y4655151dq56_3k140br0000gn/T/temporary-713a5e5e-21d3-4791-864c-63444974179c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/02 15:31:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
+-----------+
|sum(amount)|
+-----------+
+-----------+

+-------------+
|  sum(amount)|
+-------------+
|1.292712804E7|
+-------------+

+-------------------+
|        sum(amount)|
+-------------------+
|5.531697414610036E9|
+-------------------+

+-------------------+
|        sum(amount)|
+-------------------+
|5.585525162460036E9|
+-------------------+

+--------------------+
|         sum(amount)|
+------------

KeyboardInterrupt: 

In [31]:
#help(dest_count)

# window

In [23]:
#help(window)

In [21]:
# Sample Data is generated for windowing examples
windowingData = (("12", "2019-01-02 15:30:00"),
    ("12",  "2019-01-02 15:30:30"), 
    ("12",  "2019-01-02 15:31:00"), 
    ("12",  "2019-01-02 15:31:50"), 
    ("12",  "2019-01-02 15:31:55"), 
    ("16",  "2019-01-02 15:33:00"), 
    ("16",  "2019-01-02 15:35:20"), 
    ("16",  "2019-01-02 15:37:00"),
    ("20",  "2019-01-02 15:30:30"), 
    ("20",  "2019-01-02 15:31:00"), 
    ("20",  "2019-01-02 15:31:50"), 
    ("20",  "2019-01-02 15:31:55"), 
    ("20",  "2019-01-02 15:33:00"), 
    ("20",  "2019-01-02 15:35:20"), 
    ("20",  "2019-01-02 15:37:00"),  
    ("20",  "2019-01-02 15:40:00"), 
    ("20",  "2019-01-02 15:45:00"),
    ("20",  "2019-01-02 15:46:00"), 
    ("20",  "2019-01-02 15:47:30"), 
    ("20",  "2019-01-02 15:48:00"),
    ("20",  "2019-01-02 15:48:10"),
    ("20",  "2019-01-02 15:48:20"),
    ("20",  "2019-01-02 15:48:30"),
    ("20",  "2019-01-02 15:50:00"), 
    ("20",  "2019-01-02 15:53:00"),
    ("20",  "2019-01-02 15:54:30"),
    ("20",  "2019-01-02 15:55:00"),
    ("22",  "2019-01-02 15:50:30"), 
    ("22",  "2019-01-02 15:52:00"),
    ("22",  "2019-01-02 15:50:30"), 
    ("22",  "2019-01-02 15:52:00"),
    ("22",  "2019-01-02 15:50:30"), 
    ("22",  "2019-01-02 15:52:00"))
 
columns = ["eventId", "timeReceived"]
windowing_df = spark.createDataFrame(data = windowingData, schema = columns)
windowing_df.printSchema()
windowing_df.show(100, truncate=False)

root
 |-- eventId: string (nullable = true)
 |-- timeReceived: string (nullable = true)



+-------+-------------------+
|eventId|timeReceived       |
+-------+-------------------+
|12     |2019-01-02 15:30:00|
|12     |2019-01-02 15:30:30|
|12     |2019-01-02 15:31:00|
|12     |2019-01-02 15:31:50|
|12     |2019-01-02 15:31:55|
|16     |2019-01-02 15:33:00|
|16     |2019-01-02 15:35:20|
|16     |2019-01-02 15:37:00|
|20     |2019-01-02 15:30:30|
|20     |2019-01-02 15:31:00|
|20     |2019-01-02 15:31:50|
|20     |2019-01-02 15:31:55|
|20     |2019-01-02 15:33:00|
|20     |2019-01-02 15:35:20|
|20     |2019-01-02 15:37:00|
|20     |2019-01-02 15:40:00|
|20     |2019-01-02 15:45:00|
|20     |2019-01-02 15:46:00|
|20     |2019-01-02 15:47:30|
|20     |2019-01-02 15:48:00|
|20     |2019-01-02 15:48:10|
|20     |2019-01-02 15:48:20|
|20     |2019-01-02 15:48:30|
|20     |2019-01-02 15:50:00|
|20     |2019-01-02 15:53:00|
|20     |2019-01-02 15:54:30|
|20     |2019-01-02 15:55:00|
|22     |2019-01-02 15:50:30|
|22     |2019-01-02 15:52:00|
|22     |2019-01-02 15:50:30|
|22     |2

# Tumbling window

In [22]:
from pyspark.sql.functions import *

tumblingWindows = windowing_df.withWatermark("timeReceived", "10 minutes").groupBy("eventId", window("timeReceived", "10 minutes")).count()

tumblingWindows.show(truncate = False)

+-------+------------------------------------------+-----+
|eventId|window                                    |count|
+-------+------------------------------------------+-----+
|12     |{2019-01-02 15:30:00, 2019-01-02 15:40:00}|5    |
|16     |{2019-01-02 15:30:00, 2019-01-02 15:40:00}|3    |
|20     |{2019-01-02 15:30:00, 2019-01-02 15:40:00}|7    |
|20     |{2019-01-02 15:40:00, 2019-01-02 15:50:00}|8    |
|20     |{2019-01-02 15:50:00, 2019-01-02 16:00:00}|4    |
|22     |{2019-01-02 15:50:00, 2019-01-02 16:00:00}|6    |
+-------+------------------------------------------+-----+



# Sliding window

In [25]:
from pyspark.sql.functions import *

slidingWindows = windowing_df.withWatermark("timeReceived", "10 minutes").groupBy("eventId", window("timeReceived", "10 minutes", "5 minutes")).count()

slidingWindows.show(truncate = False)

+-------+------------------------------------------+-----+
|eventId|window                                    |count|
+-------+------------------------------------------+-----+
|12     |{2019-01-02 15:25:00, 2019-01-02 15:35:00}|5    |
|12     |{2019-01-02 15:30:00, 2019-01-02 15:40:00}|5    |
|16     |{2019-01-02 15:30:00, 2019-01-02 15:40:00}|3    |
|16     |{2019-01-02 15:35:00, 2019-01-02 15:45:00}|2    |
|16     |{2019-01-02 15:25:00, 2019-01-02 15:35:00}|1    |
|20     |{2019-01-02 15:25:00, 2019-01-02 15:35:00}|5    |
|20     |{2019-01-02 15:30:00, 2019-01-02 15:40:00}|7    |
|20     |{2019-01-02 15:40:00, 2019-01-02 15:50:00}|8    |
|20     |{2019-01-02 15:35:00, 2019-01-02 15:45:00}|3    |
|20     |{2019-01-02 15:45:00, 2019-01-02 15:55:00}|10   |
|20     |{2019-01-02 15:50:00, 2019-01-02 16:00:00}|4    |
|22     |{2019-01-02 15:45:00, 2019-01-02 15:55:00}|6    |
|20     |{2019-01-02 15:55:00, 2019-01-02 16:05:00}|1    |
|22     |{2019-01-02 15:50:00, 2019-01-02 16:00:00}|6   

# Session window

In [28]:
from pyspark.sql.functions import *

sessionWindows = windowing_df.withWatermark("timeReceived", "10 minutes").groupBy("eventId", session_window("timeReceived", "5 minutes")).count()

sessionWindows.show(truncate = False)

+-------+------------------------------------------+-----+
|eventId|session_window                            |count|
+-------+------------------------------------------+-----+
|12     |{2019-01-02 15:30:00, 2019-01-02 15:36:55}|5    |
|16     |{2019-01-02 15:33:00, 2019-01-02 15:42:00}|3    |
|20     |{2019-01-02 15:30:30, 2019-01-02 16:00:00}|19   |
|22     |{2019-01-02 15:50:30, 2019-01-02 15:57:00}|6    |
+-------+------------------------------------------+-----+

